In [1]:
from MyClassifier_13 import MyClassifier
import numpy as np
import requests, gzip, os, hashlib

In [6]:
### driver code
# fetching the MNIST data

### change this line of code to put the data where you want it
path = "C:/Users/AkiraSuzuki/Code"
###

def fetch(url):
    fp = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            data = f.read()
    else:
        with open(fp, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(gzip.decompress(data), dtype=np.uint8).copy()

X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 784))
y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 784))
y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

# taking only 1's and 7's
X_train = X_train[(y_train == 1) | (y_train == 7)]
y_train = y_train[(y_train == 1) | (y_train == 7)]
X_test = X_test[(y_test == 1) | (y_test == 7)]
y_test = y_test[(y_test == 1) | (y_test == 7)]


model = MyClassifier([X, y, X_test, y_test])

In [10]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(13007, 784)
(13007,)
(2163, 784)
(2163,)
